In [20]:
import os
import re
import sys
from pathlib import Path

import pandas as pd
from dotenv import load_dotenv

In [21]:
load_dotenv()                                       

DATA_DIR = os.getenv("DATA_DIR")
if DATA_DIR is None:
    sys.exit("DATA_DIR is not set.  Add it to your .env file.")

SAVE_DIR = os.getenv("SAVE_DIR")                           
if SAVE_DIR is None:
    sys.exit("SAVE_DIR is not set.  Add it to your .env file.")


save_path = Path(SAVE_DIR).expanduser()          
save_path.mkdir(parents=True, exist_ok=True) 
 

In [23]:
file_01 = base_path / "fp10_01.xlsx"
if not file_01.exists():
    sys.exit("fp10_01.xlsx not found where expected")

# List available sheet names
try:
    xls = pd.ExcelFile(file_01)
    print("Sheets in fp10_01.xlsx:", xls.sheet_names)
except Exception as exc:
    print("Could not open fp10_01.xlsx →", exc)

Sheets in fp10_01.xlsx: ['Search summary', 'Results']


In [ ]:

pattern = re.compile(r"^fp\d*_?\d+\.xlsx$")
merged_df = pd.DataFrame()

row_counts = {}

# Loop through matching files
for file in base_path.iterdir():

    if file.is_file() and pattern.match(file.name):
        try:

            df = pd.read_excel(file, sheet_name="Results")

            row_counts[file.name] = df.shape[0]

            merged_df = pd.concat([merged_df, df], ignore_index=True)

            print(f"✓ Merged: {file.name} ({df.shape[0]} rows)")
            
        except Exception as exc:

            print(f"Error reading {file.name}: {exc}")

# Save
output_file = save_path / "final_fp10_files.csv"
merged_df.to_csv(output_file, index=False)

# Prints
print(f"\nAll matched files merged and saved to: {output_file}")
print(f"Total number of rows: {merged_df.shape[0]}")

✓ Merged: fp10_02.xlsx (83419 rows)
✓ Merged: fp10_14.xlsx (66072 rows)
✓ Merged: fp10_38.xlsx (62977 rows)
✓ Merged: fp10_18.xlsx (69677 rows)
✓ Merged: fp10_34.xlsx (64037 rows)
✓ Merged: fp10_22.xlsx (69334 rows)
✓ Merged: fp10_23.xlsx (66862 rows)
✓ Merged: fp10_35.xlsx (63610 rows)
✓ Merged: fp10_19.xlsx (68039 rows)
✓ Merged: fp10_39.xlsx (63086 rows)
✓ Merged: fp10_15.xlsx (65095 rows)
✓ Merged: fp10_42.xlsx (56699 rows)
✓ Merged: fp10_03.xlsx (75376 rows)
✓ Merged: fp10_08.xlsx (66919 rows)
✓ Merged: fp10_32.xlsx (68764 rows)
✓ Merged: fp10_24.xlsx (70932 rows)
✓ Merged: fp10_04.xlsx (70883 rows)
✓ Merged: fp10_12.xlsx (66776 rows)
✓ Merged: fp10_28.xlsx (70644 rows)
✓ Merged: fp10_29.xlsx (72199 rows)
✓ Merged: fp10_13.xlsx (66102 rows)
✓ Merged: fp10_05.xlsx (72099 rows)
✓ Merged: fp10_25.xlsx (71759 rows)
✓ Merged: fp10_33.xlsx (67536 rows)
✓ Merged: fp10_09.xlsx (67291 rows)
✓ Merged: fp10_26.xlsx (69677 rows)
✓ Merged: fp10_30.xlsx (70059 rows)
✓ Merged: fp10_10.xlsx (6827

In [25]:
files = sorted(
    fp for fp in base_path.iterdir()
    if fp.is_file() and pattern.match(fp.name)
)

print(f"Found {len(files)} export files in {base_path}")

Found 42 export files in /Users/danielbivol/Library/CloudStorage/OneDrive-TechnopolisGroupLtd/Documents/data/fp10_exports


In [26]:
COL_NAME = "Company name Latin alphabet"
COL_BVD  = "BvD ID number"

summary = []
for f in files:                                         
    df = pd.read_excel(
        f,
        sheet_name="Results",
        usecols=[COL_NAME, COL_BVD],
    )
    summary.append(
        {
            "file":                      f.name,
            "unique_company_names":      df[COL_NAME].dropna().nunique(),
            "total_company_name_rows":   df[COL_NAME].notna().sum(),  
            "unique_BvD_IDs":            df[COL_BVD].dropna().nunique(),
        }
    )

summary_df = (
    pd.DataFrame(summary)
    .sort_values("file")
    .set_index("file")
)

display(summary_df)                                       

print(
    "\nPer-file totals (simple sums):\n"
    f" • Latin company names – unique: {summary_df['unique_company_names'].sum():,}\n"
    f" • Latin company names – total:  {summary_df['total_company_name_rows'].sum():,}\n"
    f" • BvD ID numbers – unique:      {summary_df['unique_BvD_IDs'].sum():,}"
)


,unique_company_names,total_company_name_rows,unique_BvD_IDs
file,,,
fp10_01.xlsx,62204,62500,62500
fp10_02.xlsx,62347,62500,62500
fp10_03.xlsx,62352,62500,62500
fp10_04.xlsx,62373,62500,62500
fp10_05.xlsx,62395,62500,62500
fp10_06.xlsx,62338,62499,62500
fp10_07.xlsx,62451,62500,62500
fp10_08.xlsx,62409,62500,62500
fp10_09.xlsx,62363,62500,62500



Per-file totals (simple sums):
 • Latin company names – unique: 2,517,063
 • Latin company names – total:  2,619,157
 • BvD ID numbers – unique:      2,618,949


In [ ]:
data_path_export01 = Path(DATA_DIR).expanduser() / "fp10_exports" 
candidates = sorted(base_path.glob("fp10_01*.xlsx"))       # fp10_01.xlsx or fp10_01_*.xlsx
if not candidates:
    raise FileNotFoundError("No file matching fp10_01*.xlsx in " + str(base_path))

fp01_path = candidates[0]
print("Using file:", fp01_path.name)

In [ ]:
df_01_export = pd.read_excel(fp01_path, sheet_name="Results")

COL = "Company name Latin alphabet"
n_unique = df_01_export[COL].dropna().nunique()

summary_df = (
    pd.DataFrame({"file": [fp01_path.name], "unique_companies": [n_unique]})
    .set_index("file")
)

display(summary_df) 